In [1]:
from utz import *
from nj_crashes.paths import ROOT_DIR
from njsp import crashes
from njsp.paths import PROJECTED_CSV
from njsp.ytc import to_ytc
from njsp.ytd import oldest_commit_rundate_since, projected_roy_deaths
from njsp import Ytd

chdir(ROOT_DIR)

In [2]:
ytd = Ytd()
prv_rundate = ytd.prv_rundate
prv_rundate

Searching for oldest commit with rundate ≥2024-09-02


Found rundate 2024-09-01 10:00:02 < 2024-09-02 at commit 6a2d7940; returning commit fcaf737


'2024-09-02 10:00:01'

In [3]:
prv_ytd_ratio = ytd.cur_ytd_deaths / ytd.prv_ytd_deaths
pct_change = (prv_ytd_ratio - 1) * 100

print(f'As of {ytd.prv_rundate}, NJSP was reporting {ytd.prv_ytd_total} YTD deaths')
print(f'Current YTD Deaths ({ytd.rundate}): {ytd.cur_ytd_deaths}')
print(f'Previous year YTD Deaths (adjusted; {ytd.prv_rundate}): {ytd.prv_ytd_deaths}')
print(f'Projected {ytd.cur_year} total: {ytd.projected_year_total}')
print(f'{pct_change:.1f}% change')

As of 2024-09-02 10:00:01, NJSP was reporting 427 YTD deaths
Current YTD Deaths (2025-09-02 10:00:01-04:00): 368
Previous year YTD Deaths (adjusted; 2024-09-02 10:00:01): 426.42490186110615
Projected 2025 total: 601.9473007430538
-13.7% change


58c53dc4e531ab8fb85c0ae42591930eaef62b50: FAUQStats cache miss: 2024, Mon Sep 02 10:00:01 EDT 2024


Find the oldest commit with rundate less than 1 year ago, load crashes from that time:

In [4]:
prv_crashes = ytd.prv_ytd_crashes
prv_crashes

,CCODE,CNAME,MCODE,MNAME,STREET,LOCATION,FATALITIES,FATAL_D,FATAL_P,FATAL_T,FATAL_B,HIGHWAY,INJURIES,dt
ACCID,,,,,,,,,,,,,,
12918,12,Middlesex,1218,Plainsboro Twsp,NaN,State Highway 1,1.0,0.0,0.0,1.0,0.0,1,NaN,2024-01-01 01:56:00-05:00
12923,11,Mercer,1111,Trenton,NaN,State Highway 129,1.0,1.0,0.0,0.0,0.0,129,NaN,2024-01-03 12:14:00-05:00
12932,01,Atlantic,0112,Hamilton Twsp,NaN,Atlantic City Expressway MP 20.5,1.0,1.0,0.0,0.0,0.0,446,0.0,2024-01-05 09:14:00-05:00
12933,02,Bergen,0248,Ramsey Boro,East Oak St,East Oak St,1.0,1.0,0.0,0.0,0.0,NaN,NaN,2024-01-05 14:12:00-05:00
13362,07,Essex,0701,Belleville Town,NaN,County 647 MP .76,1.0,0.0,0.0,1.0,0.0,647,NaN,2024-01-05 18:08:00-05:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13390,08,Gloucester,0809,Logan Twsp,NaN,Interstate 295 MP 13,1.0,0.0,0.0,1.0,0.0,295,NaN,2024-08-28 22:41:00-04:00
13394,13,Monmouth,1316,Freehold Twsp,Wemrock Rd,Wemrock Rd,1.0,0.0,1.0,0.0,0.0,NaN,NaN,2024-08-29 14:24:00-04:00
13395,17,Salem,1707,Oldmans Twsp,NaN,County 551 MP 11.15,2.0,0.0,0.0,0.0,2.0,551,0.0,2024-08-29 20:19:00-04:00


Load previous year YTD and final counts, and current year YTD counts:

In [5]:
prv_year = ytd.prv_year
cur_year = ytd.cur_year

prv_ytc = to_ytc(prv_crashes)
prv_ytd = prv_ytc.loc[prv_year]

prv_end = to_ytc(ytd.prv_end_crashes).loc[prv_year]

cur_ytc = to_ytc(ytd.cur_ytd_crashes)
cur_ytd = cur_ytc.loc[cur_year]

cur_ytd

,driver,passenger,pedestrian,cyclist,crashes
county,,,,,
Atlantic,16,2,6,0,23
Bergen,12,1,4,0,17
Burlington,16,5,4,0,23
Camden,15,2,8,1,25
Cape May,7,3,0,0,9
Cumberland,8,2,3,0,12
Essex,8,3,18,1,30
Gloucester,14,4,3,0,18
Hudson,5,1,3,1,9


Compute fraction of current year that has elapsed (year-to-date; "YTD") and still remains (rest-of-year; "ROY"):

In [6]:
cur_ytd_frac = ytd.cur_year_frac
cur_roy_frac = 1 - cur_ytd_frac
print('%.1f%% through the year, %.1f%% remaining' % (ytd.cur_year_frac * 100, ytd.cur_roy_frac * 100))

67.0% through the year, 33.0% remaining


Combine previous year YTD / end with current year YTD:

In [7]:
def melt(df, name):
    return (
        df
        .melt(ignore_index=False, var_name='type')
        .set_index('type', append=True)
        .value
        .rename(name)
    )

z = sxs(
    melt(prv_ytd, 'prv_ytd'),
    melt(prv_end, 'prv_end'),
    melt(cur_ytd, 'cur_ytd'),
).fillna(0).astype(int)
z

,,prv_ytd,prv_end,cur_ytd
county,type,,,
Atlantic,driver,20,31,16
Bergen,driver,12,14,12
Burlington,driver,20,31,16
Camden,driver,13,19,15
Cape May,driver,3,8,7
...,...,...,...,...
Salem,crashes,10,14,11
Somerset,crashes,10,18,8
Sussex,crashes,9,12,5


In [8]:
def project_roy(r):
    return int(round(projected_roy_deaths(r.prv_ytd, r.prv_end, r.cur_ytd, cur_ytd_frac)))

roy = z.apply(project_roy, axis=1).rename('roy')
projected = (
    (z.cur_ytd + roy)
    .rename('projected')
    .reset_index(level=1)
    .pivot(columns='type', values='projected')
)
projected

type,crashes,cyclist,driver,passenger,pedestrian
county,,,,,
Atlantic,41,0,26,4,12
Bergen,28,0,14,1,11
Burlington,37,0,26,5,9
Camden,35,1,22,3,10
Cape May,23,0,16,4,0
Cumberland,21,0,12,3,8
Essex,44,1,12,10,26
Gloucester,41,0,29,14,8
Hudson,18,1,9,1,8


In [9]:
projected.to_csv(PROJECTED_CSV)